In [ ]:
import plotly.graph_objects as go
import dash
from dash import dcc, html, Input, Output, State
import numpy as np
import threading
import networkx as nx
from utils import create_mesh
from head_points import find_reference_points

In [ ]:
mesh = create_mesh('data.txt')
fig = go.Figure(data=[mesh])
fig.update_layout(
    autosize=False,
    width=800,
    height=800,
)

### find reference points (at axes intersections)

In [ ]:
all_ref_points = find_reference_points(mesh)

In [ ]:
for i in all_ref_points:
    fig.add_trace(go.Scatter3d(
        x=[mesh.x[i]],
        y=[mesh.y[i]],
        z=[mesh.z[i]],
        mode='markers',
        name=f'ref point {i}',
        marker=dict(
            size=3,
            color='red',
            opacity=0.8
        )
    ))

In [ ]:
fig

# Dash app

In [ ]:
# global variable and lock for thread safety
global_click_data = None
lock = threading.Lock()

In [ ]:
app = dash.Dash()


app.layout = dash.html.Div([
    dcc.Graph(id='head-plot', figure=fig), 
    
    html.Pre(
    id='click-data',
    style={
        'backgroundColor': 'white',  # Set background color to white
        'padding': '10px',           # Optional: Add some padding
        'border': '1px solid black', # Optional: Add a border
        'borderRadius': '5px'       # Optional: Round the corners
    }
)
])

@app.callback(
    Output('head-plot', 'figure'),
    Output('click-data', 'children'),
    Input('head-plot', 'clickData'),
    State('head-plot', 'figure'),
    State('head-plot', 'relayoutData')  # Capture current view settings
)
def update_plot(clickData, existing_figure, relayoutData):
    global global_click_data
    if clickData is not None:
        with lock:
            coords = clickData['points'][0]
            global_click_data = (coords['pointNumber'], (coords['x'], coords['y'], coords['z']))

            # Replace the 'Clicked Point' in the plot
            new_point = go.Scatter3d(x=[coords['x']], y=[coords['y']], z=[coords['z']], 
                                     mode='markers', marker=dict(size=5, color='blue'),
                                     name='Clicked Point',
                                     hoverinfo='none')
            
        
            # Remove the last clicked point if it exists
            existing_figure['data'] = [trace for trace in existing_figure['data'] if trace['name'] != 'Clicked Point']
            existing_figure['data'].append(new_point)

            # Apply the captured view settings to maintain orientation
            if relayoutData and 'scene.camera' in relayoutData:
                existing_figure['layout']['scene']['camera'] = relayoutData['scene.camera']


            return existing_figure, "Clicked coordinates: x: {:.2f}, y: {:.2f}, z: {:.2f}".format(coords['x'], coords['y'], coords['z'])
    return existing_figure, "Click on a point in the plot"

app.run_server(debug=True, use_reloader=False)  # Turn off reloader if inside Jupyter

In [ ]:
global_click_data


## Calculate distance from ref points (Graph and edges only)

In [ ]:
def create_graph_from_mesh(mesh):
    G = nx.Graph()
    
    # Add vertices
    for x, y, z in zip(mesh.x, mesh.y, mesh.z):
        G.add_node((x, y, z))

    # Add edges based on faces
    for i, j, k in zip(mesh.i, mesh.j, mesh.k):
        G.add_edge((mesh.x[i], mesh.y[i], mesh.z[i]), (mesh.x[j], mesh.y[j], mesh.z[j]))
        G.add_edge((mesh.x[j], mesh.y[j], mesh.z[j]), (mesh.x[k], mesh.y[k], mesh.z[k]))
        G.add_edge((mesh.x[k], mesh.y[k], mesh.z[k]), (mesh.x[i], mesh.y[i], mesh.z[i]))

    return G

# Create the graph corresponding to the mesh
G = create_graph_from_mesh(mesh)


In [ ]:
def Euclidean_dist(G, node1, node2):
    nodes = nx.shortest_path(G, source=node1, target=node2)
    all_dists = [np.linalg.norm(np.array(nodes[i]) - np.array(nodes[i+1])) for i in range(len(nodes)-1)]

    return (sum(all_dists), nodes)

In [ ]:
# coordinates of the index 14538 (top of the head)
ref0 = mesh.x[all_ref_points[0]], mesh.y[all_ref_points[0]], mesh.z[all_ref_points[0]]
ref0

In [ ]:
app_dist = dash.Dash()


app_dist.layout = dash.html.Div([
    dcc.Graph(id='head-plot', figure=fig)
])

@app_dist.callback(
    Output('head-plot', 'figure'),
    Input('head-plot', 'clickData'),
    State('head-plot', 'figure'),
    State('head-plot', 'relayoutData')  # Capture current view settings
)
def update_figure(clickData, existing_figure, relayoutData):
    if clickData is not None:
        clicked_point = clickData['points'][0]
        coords = (clicked_point['x'], clicked_point['y'], clicked_point['z'])

        # Get the vertices to connect
        distance, vertices_to_connect = Euclidean_dist(G, coords, ref0)

        # Add the distance as an annotation near the last clicked point
        annotation = {
            'text': f"Distance: {distance:.2f}",
            'xref': 'paper',  # Use 'paper' for positioning relative to the entire plot
            'yref': 'paper',
            'x': 0.05,  # X position in paper coordinates (0 is left, 1 is right)
            'y': 0.95,  # Y position in paper coordinates (0 is bottom, 1 is top)
            'showarrow': False,  # No arrow needed
            'font': {'size': 12},
            'bgcolor': 'white',  # Background color for better visibility
            'bordercolor': 'black',
            'borderwidth': 1
        }
        existing_figure['layout']['annotations'] = [annotation]

        # Create a trace for the edges
        edge_x = []
        edge_y = []
        edge_z = []
        for i in range(len(vertices_to_connect) - 1):
            edge_x.extend([vertices_to_connect[i][0], vertices_to_connect[i + 1][0], None])
            edge_y.extend([vertices_to_connect[i][1], vertices_to_connect[i + 1][1], None])
            edge_z.extend([vertices_to_connect[i][2], vertices_to_connect[i + 1][2], None])

        edge_trace = go.Scatter3d(x=edge_x, y=edge_y, z=edge_z, mode='lines', name='edge', line=dict(color='blue', width=2))

        clicked_point_trace = go.Scatter3d(
            x=[coords[0]],
            y=[coords[1]],
            z=[coords[2]],
            mode='markers',
            marker=dict(size=5, color='blue'),
            name='Clicked Point'
        )


        # Remove the last clicked point and edges 
        existing_figure['data'] = [trace for trace in existing_figure['data'] if trace['name'] not in ['Clicked Point', 'edge']]
        existing_figure['data'].append(clicked_point_trace)
        existing_figure['data'].append(edge_trace)

        # Apply the captured view settings to maintain orientation
        if relayoutData and 'scene.camera' in relayoutData:
            existing_figure['layout']['scene']['camera'] = relayoutData['scene.camera']

        
        return existing_figure
    return existing_figure

app_dist.run_server(debug=True, use_reloader=False)  # Turn off reloader if inside Jupyter

# App with potpourri3d

In [ ]:
import potpourri3d as pp3d
import trimesh as tm

In [ ]:

vertices = np.column_stack((mesh.x, mesh.y, mesh.z))
faces = np.column_stack((mesh.i, mesh.j, mesh.k))

# Create a Trimesh object
mesh_tm = tm.Trimesh(vertices=vertices, faces=faces)

# Merge duplicate vertices
mesh_tm.merge_vertices()

# Export to PLY
mesh_tm.export('data.ply')

In [ ]:
mesh_tm

In [ ]:
V, F = pp3d.read_mesh('data.ply')

In [ ]:
path_solver = pp3d.EdgeFlipGeodesicSolver(V,F) # shares precomputation for repeated solves
path_pts = path_solver.find_geodesic_path(v_start=14, v_end=22)
# path_pts is a Vx3 numpy array of points forming the path